In [1]:
from pipeline.util.platform import start_spark, start_spark_local
from pipeline.util.storage import read_csv, read_parquet, write_parquet, write_repart_parquet, write_delta, write_deltatable
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [2]:
spark, loggrer, conf = start_spark(app_name='addessmatch_etl_job')

local


In [3]:
df_bld = read_parquet(spark, "{storage_curated}/data/in/addressmatchres".format(storage_curated = spark.conf.get('storage.curated')))

In [5]:
df_bld_id = df_bld.withColumn("BUILDING_ID", F.monotonically_increasing_id() + F.unix_timestamp(F.current_timestamp()))

In [6]:
df_bld_id.createOrReplaceTempView("building_in")

In [42]:
loc = "'{storage_curated}/data/out/building'".format(storage_curated = spark.conf.get('storage.curated'))

#spark.sql(f"DROP TABLE IF EXISTS BUILDING")
spark.sql(f"CREATE TABLE IF NOT EXISTS BUILDING ( \
          ID INT, \
          BUILDINGADDRESS STRING, \
          BUILDING_ID BIGINT, \
          ADDRESS_ID BIGINT ) \
          USING DELTA  \
          LOCATION {loc}")

DataFrame[]

In [47]:
spark.sql(f"MERGE INTO BUILDING \
          USING ( \
            SELECT \
              b.id, \
              b.buildingaddress, \
              b.BUILDING_ID, \
              a.ADDRESS_ID \
            FROM building_in b \
            JOIN ADDRESS a ON b.UPRN = a.UPRN \
          ) AS building_in \
          ON BUILDING.ID = building_in.id \
          WHEN MATCHED THEN \
            UPDATE SET \
              BUILDINGADDRESS = building_in.buildingaddress, \
              ADDRESS_ID = building_in.ADDRESS_ID \
          WHEN NOT MATCHED THEN \
            INSERT ( \
              ID, BUILDINGADDRESS, BUILDING_ID, ADDRESS_ID \
            ) VALUES ( \
              building_in.id, building_in.buildingaddress, building_in.BUILDING_ID, building_in.ADDRESS_ID \
            )")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [50]:
#df_admatchres.printSchema()
#spark.sql("describe table building_in").show()
#spark.sql("select * from BUILDING order by ID").show()
#spark.sql("delete from BUILDING where ID > 10")
#spark.sql("select * from BUILDING a join ADDRESS b on a.ADDRESS_ID=b.ADDRESS_ID order by a.ID").show(truncate=False)
#spark.sql("show tables").show()
#spark.sql("drop table addressmatchres")
#spark.sql(f"describe history {loc}").show(truncate=False)
#spark.sql("select * from BUILDING version as of 5").show(truncate=False)

+---+--------------------------------------------------------------------------+-----------+-----------+------------+-------+--------+---------------------------------------------------------------+-----------+
|ID |BUILDINGADDRESS                                                           |BUILDING_ID|ADDRESS_ID |UPRN        |OUTCODE|LOCALITY|ADDRESS                                                        |ADDRESS_ID |
+---+--------------------------------------------------------------------------+-----------+-----------+------------+-------+--------+---------------------------------------------------------------+-----------+
|1  |The South Lawn Medical Practice, S Lawn Terrace, Heavitree, Exeter EX1 2RX|1710870608 |18890606528|100041046145|2RX    |null    |HEAVITREE HEALTH CENTRE SOUTH LAWN TERRACE EXETER DEVON EX1 2RX|18890606528|
|4  |FLAT 52D MORTIMER HOUSE GRENDON ROAD EX1 2NL                              |1710870609 |10300663509|100040215247|2NL    |null    |FLAT 52D MORTIMER HOUS